In [2]:
#insatll the nlputils library
! pip install nlputils

In [3]:
#Import necessary libraries
import nlputils as nu
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [4]:
#upload train data
#upload textfile into colab
#from google.colab import files

#uploaded = files.upload()

#for fn in uploaded.keys():
 # print('User uploaded file "{name}" with length {length} bytes'.format(
  #    name=fn, length=len(uploaded[fn])))

In [5]:
!gdown --id 1L6Xxa1XbyRs-7P8pBw3EakOwi8o1loME

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1L6Xxa1XbyRs-7P8pBw3EakOwi8o1loME
To: /content/train.csv
100% 98.6M/98.6M [00:00<00:00, 197MB/s]


In [6]:
#read the uploaded file
#import io
#Raw_data = pd.read_csv(io.BytesIO(uploaded['train.csv']))
#explore the raw data DF
#Raw_data.head()


In [7]:
#read the imported file
Raw_data=pd.read_csv('train.csv')
#Explore the raw data DF
Raw_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
#Data exploration
#checking for shape
Raw_data.shape

(20800, 5)

In [9]:
#checking for null 
Raw_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
#dropping the null values
df=Raw_data.dropna()


In [11]:
#checking for null values in the new data frame
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [12]:
df.reset_index(inplace=True)

In [13]:
#shape of new dataframe
df.shape

(18285, 6)

In [14]:
#explore cleaned DF
df.head()

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [15]:
#expand column width
pd.set_option('display.max_colwidth',None)

In [16]:
#explore the title column
df['title'].head()

0                House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It
1                                          FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart
2                                                                Why the Truth Might Get You Fired
3                                  15 Civilians Killed In Single US Airstrike Have Been Identified
4    Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery
Name: title, dtype: object

In [17]:
#explore the text column
df['text'].head()

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

Cleaning Text

In [18]:
#import regular expressions and string
import string
import re

In [19]:
#convert your text to lower case using a lambda function
# .lower() - convert all strings to lowercase 
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

In [20]:
#remove \n from the text and replacing with ""
remove_n = lambda x: re.sub("\n", " ", x)

In [21]:
#remove all non ascii characters
removenonascii=lambda x: re.sub(r'[^\x00-\x7f]',r' ',x) 


In [22]:
#remove alphanumeric values
remove_alphanumeric=lambda x: re.sub('\w*\d\w*', ' ', x)

In [23]:
#use the map function to apply the lambda functions
# Apply all the lambda functions wrote previously through .map on the comments column
df['text'] = df['text'].map(remove_alphanumeric).map(punc_lower).map(remove_n).map(removenonascii)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
#explore the text column
df['text'].head()

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

## Remove stop words

Use the porters stemming algorithm. The Porter stemming algorithm (or 'Porter stemmer') is a process for removing the commoner morphological and inflexional endings from words in English. Its main use is as part of a term normalisation process that is usually done when setting up Information Retrieval systems.

In [25]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [26]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0, len(df)):
  review=re.sub('[^a-zA-Z]',' ',df['text'][i])
  review=review.lower()
  review=review.split()
  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

In [27]:
#convert all items in corpus to a string
corpus = [str (item) for item in corpus]

Splitting data farme and creating the train and test set


In [28]:
y=df['label']
y.shape

(18285,)

In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(df['text'], y, test_size=0.30, random_state=40)
## We have split the data into 70 percent train and 30 percent test

Our data set is already split into train and test data. No need to apply split in this case. MOve on to the vectorizer. Can use either tfidf or count vectorizer.

In [30]:
#Applying tfidf to the data set
tfidf_vect = TfidfVectorizer(stop_words = 'english',max_df=0.7)
tfidf_train = tfidf_vect.fit_transform(X_train)
tfidf_test = tfidf_vect.transform(X_test)

In [31]:
print(tfidf_train.shape)

(12799, 117046)


In [32]:
Y_train.shape

(12799,)

In [33]:
print(tfidf_test)

  (0, 116553)	0.03886548223169257
  (0, 115702)	0.02577799575708681
  (0, 115692)	0.00904800329154621
  (0, 114895)	0.009673419081344278
  (0, 114861)	0.016618616020114874
  (0, 114848)	0.010562420147987915
  (0, 113165)	0.013935134858730162
  (0, 113110)	0.016117563226566926
  (0, 113013)	0.027915866741656793
  (0, 112960)	0.0187879083094958
  (0, 112621)	0.014233357454810993
  (0, 112502)	0.033848840233184616
  (0, 112472)	0.018913078792303722
  (0, 111638)	0.022574963902960127
  (0, 111149)	0.03305268263185922
  (0, 110726)	0.030170078331103658
  (0, 110466)	0.011808060706868613
  (0, 110343)	0.03305268263185922
  (0, 110196)	0.017882582744640885
  (0, 108797)	0.0201639418741461
  (0, 107753)	0.0445860293134556
  (0, 107752)	0.0445860293134556
  (0, 107301)	0.026722385232109524
  (0, 106653)	0.04142088061358076
  (0, 106336)	0.015087881216580364
  :	:
  (5485, 14393)	0.030676892638861704
  (5485, 13981)	0.03850878930619704
  (5485, 13630)	0.05358526389326042
  (5485, 11899)	0.037197

In [34]:
#get feature names of the tfidf vectorizer
print(tfidf_vect.get_feature_names()[-12:])

['zwolennikami', 'zwozdiak', 'zwrocie', 'zwrotu', 'zwyci', 'zxycmwjdxj', 'zyaratgah', 'zygankow', 'zylinderkopfdichtung', 'zynga', 'zytsov', 'zyuganov']


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [35]:
#applying count_vectorizer
#applying tfidf to the dataset
#count_vect=CountVectorizer(stopwords='english')
#count_train=CountVectorizer.fit_transform(df['text'].values)

In [36]:
#print(count_train)

In [37]:
#get feature names of the tfidf vectorizer
#print(count_vect.get_feature_names()[-12:])

K-Nearest Neighbor Model

In [38]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [39]:
#fit the model on train data 
KNN = KNeighborsClassifier().fit(tfidf_train,Y_train)
#predict on train 
train_preds4 = KNN.predict(tfidf_train)
#accuracy on train
print("Model accuracy on train is: ", accuracy_score(Y_train, train_preds4))

#predict on test
test_preds4 = KNN.predict(tfidf_test)
#accuracy on test
print("Model accuracy on test is: ", accuracy_score(Y_test, test_preds4))
print('-'*50)

Model accuracy on train is:  0.5187123994062036
Model accuracy on test is:  0.48651111921254103
--------------------------------------------------


In [40]:
#Confusion matrix
print("confusion_matrix train is: ", confusion_matrix(Y_train, train_preds4))
print("confusion_matrix test is: ", confusion_matrix(Y_test, test_preds4))
print('Wrong predictions out of total')
print('-'*50)

# Wrong Predictions made.
print((Y_test !=test_preds4).sum(),'/',((Y_test == test_preds4).sum()+(Y_test != test_preds4).sum()))

print('-'*50)

confusion_matrix train is:  [[1106 6144]
 [  16 5533]]
confusion_matrix test is:  [[ 304 2807]
 [  10 2365]]
Wrong predictions out of total
--------------------------------------------------
2817 / 5486
--------------------------------------------------
